In [5]:
import pandas as pd
import numpy as np
import json

In [6]:
with open("cgam_ebs.json", "r") as f:
    result = json.load(f)


dfs_ebs = {}
for key in ["components", "connections"]:
    dfs_ebs[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

with open("cgam_tespy.json", "r") as f:
    result = json.load(f)


dfs_tespy = {}
for key in ["components", "connections"]:
    dfs_tespy[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [7]:
dfs_ebs["connections"][["e_T", "e_M"]] = dfs_ebs["connections"][["e_T", "e_M"]].round(6)
dfs_tespy["connections"][["e_T", "e_M"]] = dfs_tespy["connections"][["e_T", "e_M"]].round(6)

In [8]:
columns = ["m", "p", "T", "e_T", "e_M"]

overlapping_index = list(set(dfs_tespy[key].index.tolist()) & set(dfs_ebs[key].index.tolist()))
diff_to_tespy = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ) / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)
diff_to_ebs = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ).abs() / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)

In [9]:
diff_to_ebs

,m,p,T,e_T,e_M
3,0.000183,5.195615e-05,2.674984e-16,1.337030e-03,0.000499
7,0.000155,0.000000e+00,1.002291e-03,4.902781e-03,NaN
4,0.000155,3.554894e-05,6.580088e-08,8.278086e-04,0.000539
2,0.000183,2.298427e-16,8.233857e-04,5.170441e-03,0.000541
9,0.000000,0.000000e+00,2.822459e-10,2.908972e-10,0.000000
6,0.000155,2.961505e-04,7.014391e-04,1.462352e-03,0.005671
8P,0.000000,0.000000e+00,1.571228e-12,1.921363e-11,0.000000
10,0.001398,0.000000e+00,6.539424e-14,NaN,0.007728
6P,0.000155,1.769711e-04,8.883634e-04,2.936085e-03,0.006993
5,0.000155,2.680162e-04,3.328763e-04,3.284637e-04,0.003172


In [10]:
diff_to_ebs[diff_to_ebs > 1e-3].dropna(how="all")

,m,p,T,e_T,e_M
3,NaN,NaN,NaN,0.001337,NaN
7,NaN,NaN,0.001002,0.004903,NaN
2,NaN,NaN,NaN,0.005170,NaN
6,NaN,NaN,NaN,0.001462,0.005671
10,0.001398,NaN,NaN,NaN,0.007728
6P,NaN,NaN,NaN,0.002936,0.006993
5,NaN,NaN,NaN,NaN,0.003172


In [11]:
# Load the CSV files
tespy_results = pd.read_csv("cgam_components_tespy.csv", index_col=0)
ebsilon_results = pd.read_csv("cgam_components_ebsilon.csv", index_col=0)

# Step 1: Normalize the component names in Tespy
# Group "drum" and "drum pump" into "EVA"
tespy_results.loc[tespy_results["Component"].isin(["drum"]), "Component"] = "EV"

# Aggregate the values by summing them up
tespy_results = tespy_results.groupby("Component", as_index=False).sum(numeric_only=True)
ebsilon_results = ebsilon_results.groupby("Component", as_index=False).sum(numeric_only=True)

# Step 2: Merge DataFrames on Component name
merged_df = pd.merge(ebsilon_results, tespy_results, on="Component", suffixes=("_Ebsilon", "_Tespy"))

# Step 3: Compute Differences
columns_to_compare = ["E_F [kW]", "E_P [kW]", "E_D [kW]", "E_L [kW]", "ε [%]", "y [%]", "y* [%]"]
for col in columns_to_compare:
    merged_df[f"Diff_{col}"] = merged_df[f"{col}_Tespy"] - merged_df[f"{col}_Ebsilon"]

# Step 4: Select only the relevant columns
diff_columns = [col for col in merged_df.columns if col.startswith("Diff_E_D") or col.endswith("E_D [kW]_Ebsilon") or col.endswith("E_D [kW]_Tespy")]
diff_df = merged_df[["Component"] + diff_columns]
diff_df["Relative error [%]"] = diff_df["Diff_E_D [kW]"] / diff_df["E_D [kW]_Tespy"] * 100

# Display the filtered DataFrame
diff_df

C:\Users\sergiotomasinelli\AppData\Local\Temp\ipykernel_15608\3706457234.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_df["Relative error [%]"] = diff_df["Diff_E_D [kW]"] / diff_df["E_D [kW]_Tespy"] * 100


,Component,E_D [kW]_Ebsilon,E_D [kW]_Tespy,Diff_E_D [kW],Relative error [%]
0,AC,2100.488431,2101.222024,0.733594,0.034913
1,APH,2559.563096,2557.498135,-2.064962,-0.080741
2,CC,25314.346057,25397.802723,83.456666,0.328598
3,EV,4586.829577,4600.584989,13.755412,0.298993
4,EXP,2994.122460,2995.010710,0.888250,0.029658
5,PH,1911.387859,1915.951499,4.563640,0.238192
6,TOT,39466.737486,39568.070080,101.332593,0.256097
